# Práctica 4: Detección de características 

## Participantes:
- Gerardo León Quintana
- Susana Suárez Mendoza

## Ejercicio 1: 

Desarrolle una aplicación que permita:

a. A través de la interfaz modificar los parámetros del detector de características SIFT.

b. Seleccionar un área de interés en una imagen de elección una imagen de naturaleza médica y una imagen telemétrica. 

c. Buscar esa área de interés (recuádrela en rojo) dentro de diferentes versiones de la imagen de partida (con cambios de traslación, escala y rotación) [NOTA: Estos cambios se pueden acometer con un editor de imágenes o con el trabajo hecho en prácticas previas]. Altere mediante la interfaz las configuraciones de parámetros para mejorar la detección. 

d. Pruebe a hacer lo mismo que en el apartado c. con diferentes grados de deformación de la imagen. [NOTA: Estos cambios se pueden acometer con un editor de imágenes o con el trabajo hecho en prácticas previas].

(OPICIONAL)
A partir del detector de características “corrija” la imagen alterada para que se alinee con la imagen fuente. 
NOTA 1: Si no encuentra el área de interés la aplicación debería notificarlo. 
NOTA 2: Puede preprocesar la imagen con alguna técnica (suavizado, contrastado,…) si lo estima necesario.